In [23]:
# pip als Paketmanager
#! pip install -q pyscipopt
#! pip install pandas
#!pip install openpyxl

In [24]:
from pyscipopt import Model, quicksum

**Optimierungsmodell für den Kauf und Verkauf von Strom auf dem Strommarkt**

In [25]:
# Erstellen einer Modellinstanz
scip = Model()

**Indexmenge**

In [26]:
H = [n for n in range(1, 25)]
R = [r for r in range(0, 25)]
# Erklärung:
# es gibt 24 Risikoabstufungen, höchstes R -> höchstes Risiko, r=0, kein Risiko
# hohes Risiko => hohe Wahrscheinlichkeit gemäß der Prognosetabelle, den Zuschlag nicht zu bekommen
# Je höher die Risikoklasse ist, desto vorteilhafter (niedriger bei Kauf und höher bei Verkauf) ist der Preis, den man als Gebotspreis abgibt
# Risikoklasse 0 = kein Risiko -> normaler Erwartungswert
# Risikoklasse 1 = 4% "Risiko" und kalkulatorischer Erwartungswert wird berechnet wie folgt: 
    # 96% Eintrittswahrscheinlichkeit -> Erwartungswert aus den nicht durch das Gebot ausgeschlossenen Preismöglichkeit
    # 4% Eintrittswahrscheinlichkeit -> Erwartungswert = Moving Average der 3 umliegenden Zeitpunkte verrechnet mit penalty
    # kalkulatorischer Erwartungswert ist der gewichtete Durchschnitt nach Eintrittswahrscheinlichkeit

# Risikowahrscheinlichkeiten, diese sind nötig, um später die gewichteten Durchschnitte zu bilden
prob_r = {}
for r in R:
    prob_r[r] = 1 - r/25

print(prob_r)

{0: 1.0, 1: 0.96, 2: 0.92, 3: 0.88, 4: 0.84, 5: 0.8, 6: 0.76, 7: 0.72, 8: 0.6799999999999999, 9: 0.64, 10: 0.6, 11: 0.56, 12: 0.52, 13: 0.48, 14: 0.43999999999999995, 15: 0.4, 16: 0.36, 17: 0.31999999999999995, 18: 0.28, 19: 0.24, 20: 0.19999999999999996, 21: 0.16000000000000003, 22: 0.12, 23: 0.07999999999999996, 24: 0.040000000000000036}


**Batterie-Systemspezifikationen**

In [27]:
fixe_zykluskosten = False

wirkungsgrad_wechselrichter = 0.985
wirkungsgrad_laden = 0.975
round_trip_efficiency = 0.95
entlade_verlust = wirkungsgrad_laden - round_trip_efficiency

wirkungsgrad_systemeingang = wirkungsgrad_wechselrichter * wirkungsgrad_laden
wirkungsgrad_systemausgang = (1-(entlade_verlust / wirkungsgrad_laden)) * wirkungsgrad_wechselrichter


f_e = wirkungsgrad_systemeingang # Faktor Einkauf
f_v = wirkungsgrad_systemausgang # Faktor Verkauf


nennkapazität = 40 # MWh brutto
lademinimum = 0.2 # 20%
lademaximum = 1 # 100%
anfangsbestand = 0.5 # 50%

nettokapazität = zyklus = nennkapazität * (lademaximum - lademinimum) # MWh netto
zykluskosten = 1500 # € / zyklus
mwh_zykluskosten = zykluskosten / zyklus # € / MWh -> entweder geladen oder entladen

erlaubte_zyklen_pro_tag = 2

a = anfangsbestand * nennkapazität # MWh Anfangs- und Endbestand
u = lademinimum * nennkapazität # MWh Untergrenze Batteriekapazität
o = lademaximum * nennkapazität # MWh Obergrenze Batteriekapazität

c = 0.5 # nennkapazität / h

# Prozentuale penalty, wenn nicht Zuschlag
penalty = 2 # 200%


# Sicherstellen, dass unsere Faktoren für Systemeingang und -ausgang den multiplizierten Wirkungsgraden entspricht
print(wirkungsgrad_systemeingang*wirkungsgrad_systemausgang)
print(wirkungsgrad_wechselrichter*wirkungsgrad_wechselrichter*round_trip_efficiency)

0.92171375
0.92171375


**Vorhersagedaten**

In [28]:
import pandas as pd

prognose = pd.read_excel('Preisprognosen.xlsx')


# Erwartungswerte nach Risikoklasse und Stunde für Einkauf

preis_verkauf_h_r = {}
preis_einkauf_h_r = {}

gebotspreis_verkauf_h_r = {}
gebotspreis_einkauf_h_r = {}


for h in H:
  # Berechnung von "Standard-EW" für Stunde h
  stundenprognose = prognose[prognose['Stunde'] == h]
  # Sortiere stundenprognose nach dem Wert in Spalte "Strompreis"
  stundenprognose = stundenprognose.sort_values(by='Strompreis')
  # Index zurücksetzen
  stundenprognose = stundenprognose.reset_index(drop=True)
  for r in R:
    # Nehme den Durschnitt von der Spalte "Strompreis" vom Index r bis zum höchsten Index

    preis_verkauf_h_r[(h, r)] = stundenprognose.loc[r:24, 'Strompreis'].mean()
    gebotspreis_verkauf_h_r[(h, r)] = stundenprognose.loc[r:24, 'Strompreis'].min()
    
    preis_einkauf_h_r[(h, r)] = stundenprognose.loc[0:24-r, 'Strompreis'].mean()
    gebotspreis_einkauf_h_r[(h, r)] = stundenprognose.loc[0:24-r, 'Strompreis'].max()


p_h = {}


print(preis_verkauf_h_r)
print(preis_einkauf_h_r)


# Für die Berechnung des Vorfaktors derPenalty möchten wir den Durchschnittspreis der umliegenden Stunden zu jeder Stunde haben
moving_average_h = {}
for h in H:
    stundenprognose = prognose[prognose['Stunde'].isin([h-1,h,h+1])]
    moving_average_h[h] = stundenprognose['Strompreis'].mean()

print(moving_average_h)

{(1, 0): np.float64(81.52), (1, 1): np.float64(83.54166666666667), (1, 2): np.float64(85.21739130434783), (1, 3): np.float64(86.77272727272727), (1, 4): np.float64(88.23809523809524), (1, 5): np.float64(89.65), (1, 6): np.float64(91.05263157894737), (1, 7): np.float64(92.44444444444444), (1, 8): np.float64(93.82352941176471), (1, 9): np.float64(95.1875), (1, 10): np.float64(96.6), (1, 11): np.float64(98.0), (1, 12): np.float64(99.46153846153847), (1, 13): np.float64(100.91666666666667), (1, 14): np.float64(102.45454545454545), (1, 15): np.float64(104.1), (1, 16): np.float64(105.77777777777777), (1, 17): np.float64(107.625), (1, 18): np.float64(109.57142857142857), (1, 19): np.float64(111.66666666666667), (1, 20): np.float64(114.0), (1, 21): np.float64(116.75), (1, 22): np.float64(120.0), (1, 23): np.float64(124.0), (1, 24): np.float64(130.0), (2, 0): np.float64(72.24), (2, 1): np.float64(74.08333333333333), (2, 2): np.float64(75.6086956521739), (2, 3): np.float64(77.0), (2, 4): np.floa

**Entscheidungsvariablen**

In [29]:
# Für jede Risikoklasse und jede Stunde eine binäre Variable
bin_h_r={}
# Für jede Risikoklasse und jede Stunde eine Mengen-Variable
e_h_r={}
v_h_r={}

for h in H:
    for r in R:
      e_h_r[(h, r)] = scip.addVar(vtype='C', lb=0, ub=None, name=f"e_{h}_{r}")
      v_h_r[(h, r)] = scip.addVar(vtype='C', lb=0, ub=None, name=f"v_{h}_{r}")


print('Entscheidungsvariablen =', scip.getVars())

Entscheidungsvariablen = [e_1_0, v_1_0, e_1_1, v_1_1, e_1_2, v_1_2, e_1_3, v_1_3, e_1_4, v_1_4, e_1_5, v_1_5, e_1_6, v_1_6, e_1_7, v_1_7, e_1_8, v_1_8, e_1_9, v_1_9, e_1_10, v_1_10, e_1_11, v_1_11, e_1_12, v_1_12, e_1_13, v_1_13, e_1_14, v_1_14, e_1_15, v_1_15, e_1_16, v_1_16, e_1_17, v_1_17, e_1_18, v_1_18, e_1_19, v_1_19, e_1_20, v_1_20, e_1_21, v_1_21, e_1_22, v_1_22, e_1_23, v_1_23, e_1_24, v_1_24, e_2_0, v_2_0, e_2_1, v_2_1, e_2_2, v_2_2, e_2_3, v_2_3, e_2_4, v_2_4, e_2_5, v_2_5, e_2_6, v_2_6, e_2_7, v_2_7, e_2_8, v_2_8, e_2_9, v_2_9, e_2_10, v_2_10, e_2_11, v_2_11, e_2_12, v_2_12, e_2_13, v_2_13, e_2_14, v_2_14, e_2_15, v_2_15, e_2_16, v_2_16, e_2_17, v_2_17, e_2_18, v_2_18, e_2_19, v_2_19, e_2_20, v_2_20, e_2_21, v_2_21, e_2_22, v_2_22, e_2_23, v_2_23, e_2_24, v_2_24, e_3_0, v_3_0, e_3_1, v_3_1, e_3_2, v_3_2, e_3_3, v_3_3, e_3_4, v_3_4, e_3_5, v_3_5, e_3_6, v_3_6, e_3_7, v_3_7, e_3_8, v_3_8, e_3_9, v_3_9, e_3_10, v_3_10, e_3_11, v_3_11, e_3_12, v_3_12, e_3_13, v_3_13, e_3_14, v_

# **Zielfunktion**

In [30]:
gewinn_kauf_verkauf = quicksum(((preis_verkauf_h_r[(h, r)] * v_h_r[(h, r)] * prob_r[r] +\
                                 moving_average_h[h] * v_h_r[(h, r)] * (1-prob_r[r]) * 1/penalty) -\

                                (preis_einkauf_h_r[(h, r)] * e_h_r[(h, r)] * prob_r[r] +\
                                 moving_average_h[h] * e_h_r[(h, r)] * (1-prob_r[r]) * penalty ))\

                               for r in R for h in H )

if fixe_zykluskosten:
    zykluskosten = 3000
else:
    zykluskosten = quicksum((e_h_r[(h,r)] * f_e * mwh_zykluskosten) for h in H for r in R)

scip.setObjective(gewinn_kauf_verkauf - zykluskosten, sense="maximize")
print(scip.getObjective())

Expr({Term(e_1_0): -126.537578125, Term(v_1_0): 81.52, Term(e_1_1): -127.487978125, Term(v_1_1): 81.7376, Term(e_1_2): -128.918378125, Term(v_1_2): 81.4752, Term(e_1_3): -130.58877812499998, Term(v_1_3): 80.97279999999999, Term(e_1_4): -132.459178125, Term(v_1_4): 80.27040000000001, Term(e_1_5): -134.48957812499998, Term(v_1_5): 79.40800000000002, Term(e_1_6): -136.639978125, Term(v_1_6): 78.4256, Term(e_1_7): -138.91037812500002, Term(v_1_7): 77.3232, Term(e_1_8): -141.30077812500002, Term(v_1_8): 76.10079999999999, Term(e_1_9): -143.811178125, Term(v_1_9): 74.7584, Term(e_1_10): -146.40157812500001, Term(v_1_10): 73.336, Term(e_1_11): -149.11197812499998, Term(v_1_11): 71.7936, Term(e_1_12): -151.90237812499998, Term(v_1_12): 70.1712, Term(e_1_13): -154.772778125, Term(v_1_13): 68.4288, Term(e_1_14): -157.76317812500002, Term(v_1_14): 66.6064, Term(e_1_15): -160.83357812499997, Term(v_1_15): 64.704, Term(e_1_16): -164.02397812499999, Term(v_1_16): 62.681599999999996, Term(e_1_17): -1

***Nebenbedingungen/ Restriktionen***

In [31]:
# Ladestand zur Stunde 0 = Ladestand zur Stunde 24, also Summe Lademenge und Entlademenge gleich
scip.addCons(quicksum(((e_h_r[(h,r)] * f_e) - (v_h_r[(h,r)] / f_v)) for r in R for h in H) == 0, name="Anfangs- und Endbestand gleich")

# Maximale Ladezyklen am pro Tag anhand der Einkaufsmenge (mit Faktor = Lademenge), alternativ anhand der Verkaufsmenge
scip.addCons(quicksum((e_h_r[(h,r)] * f_e) for r in R for h in H) <= (erlaubte_zyklen_pro_tag * nettokapazität), name="Maximale Ladezyklen pro Tag")

# Mindestladestand nicht unterschritten und Höchstladestand nicht überschritten
for h in H:
    H_t =  [n for n in range(1, h+1)]
    scip.addCons( (a + quicksum(((e_h_r[(t, r)] * f_e) - (v_h_r[(t, r)] / f_v)) for r in R for t in H_t)) >= u, name=f"Mindestladestand zum Zeitpunkt t={h}")
    scip.addCons( (a + quicksum(((e_h_r[(t, r)] * f_e) - (v_h_r[(t, r)] / f_v)) for r in R for t in H_t)) <= o, name=f"Maximalladestand zum Zeitpunkt t={h}")

# Lade- und Entladeleistung begrenzt (C-Rate)
for h in H:
    scip.addCons(quicksum(((e_h_r[(h, r)] * f_e) + (v_h_r[(h, r)] / f_v)) for r in R) <= c * nennkapazität, name=f"Lade-/Entladeleistung der Stunde h={h}")


print('Nebenbedingungen =', scip.getConss())

Nebenbedingungen = [Anfangs- und Endbestand gleich, Maximale Ladezyklen pro Tag, Mindestladestand zum Zeitpunkt t=1, Maximalladestand zum Zeitpunkt t=1, Mindestladestand zum Zeitpunkt t=2, Maximalladestand zum Zeitpunkt t=2, Mindestladestand zum Zeitpunkt t=3, Maximalladestand zum Zeitpunkt t=3, Mindestladestand zum Zeitpunkt t=4, Maximalladestand zum Zeitpunkt t=4, Mindestladestand zum Zeitpunkt t=5, Maximalladestand zum Zeitpunkt t=5, Mindestladestand zum Zeitpunkt t=6, Maximalladestand zum Zeitpunkt t=6, Mindestladestand zum Zeitpunkt t=7, Maximalladestand zum Zeitpunkt t=7, Mindestladestand zum Zeitpunkt t=8, Maximalladestand zum Zeitpunkt t=8, Mindestladestand zum Zeitpunkt t=9, Maximalladestand zum Zeitpunkt t=9, Mindestladestand zum Zeitpunkt t=10, Maximalladestand zum Zeitpunkt t=10, Mindestladestand zum Zeitpunkt t=11, Maximalladestand zum Zeitpunkt t=11, Mindestladestand zum Zeitpunkt t=12, Maximalladestand zum Zeitpunkt t=12, Mindestladestand zum Zeitpunkt t=13, Maximallades

**Berechnung der Lösung**

In [32]:
scip.setIntParam("display/verblevel", 5)  # Set verbosity level to 5


scip.optimize()
# Status des Solvers
status = scip.getStatus()
print(f"Status des Solvers: {status} \n")

if status == "optimal":
    print('LÖSUNG:')
    print('Zielfunktionswert (Gewinn) =', scip.getObjVal())
    for h in H:
      v_h = 0
      e_h = 0
      risikoklasse_v = None
      risikodict_v = {}
      risikoklasse_e = None
      risikodict_e = {}
      for r in R:
        risikodict_v[r] = scip.getVal(v_h_r[(h, r)])
        risikodict_e[r] = scip.getVal(e_h_r[(h, r)])
        if scip.getVal(v_h_r[(h, r)]) > 0:
          risikoklasse_v = r

        if scip.getVal(e_h_r[(h, r)]) > 0:
          risikoklasse_e = r

        v_h += scip.getVal(v_h_r[(h, r)])
        e_h += scip.getVal(e_h_r[(h, r)])
      print("EINKAUF Stunde", h, " : " , e_h)
      if risikoklasse_e != None: print("EINKAUF Risikoklasse Stunde", h, " : ", risikoklasse_e)
      #print("EINKAUF Risiko: ", risikodict_e)
      print("Verkauf Stunde", h, " : " , v_h)
      if risikoklasse_v != None: print("VERKAUF Risikoklasse Stunde", h, " : ", risikoklasse_v)
      #print("VERKAUF Risiko: ", risikodict_v)
else:
    print('Problem hat keine Lösung')

LP Solver <Soplex 7.1.1>: barrier convergence tolerance cannot be set -- tolerance of SCIP and LP solver may differ
LP Solver <Soplex 7.1.1>: fastmip setting not available -- SCIP parameter has no effect
LP Solver <Soplex 7.1.1>: number of threads settings not available -- SCIP parameter has no effect
Status des Solvers: optimal 

LÖSUNG:
Zielfunktionswert (Gewinn) = 2147.819629826543
EINKAUF Stunde 1  :  0.0
Verkauf Stunde 1  :  0.0
EINKAUF Stunde 2  :  0.0
Verkauf Stunde 2  :  0.0
EINKAUF Stunde 3  :  0.0
Verkauf Stunde 3  :  0.0
EINKAUF Stunde 4  :  0.0
Verkauf Stunde 4  :  0.0
EINKAUF Stunde 5  :  0.0
Verkauf Stunde 5  :  0.0
EINKAUF Stunde 6  :  0.0
Verkauf Stunde 6  :  0.0
EINKAUF Stunde 7  :  0.0
Verkauf Stunde 7  :  0.0
EINKAUF Stunde 8  :  0.0
Verkauf Stunde 8  :  11.516923076923078
VERKAUF Risikoklasse Stunde 8  :  3
EINKAUF Stunde 9  :  0.0
Verkauf Stunde 9  :  0.0
EINKAUF Stunde 10  :  0.0
Verkauf Stunde 10  :  -1.7048470897115737e-15
EINKAUF Stunde 11  :  0.0
Verkauf Stund

In [33]:
# Ergebnisse in Excel speichern

import pandas as pd
from openpyxl import load_workbook


# File and worksheet details
file_path = "Ergebnisse.xlsx"
if fixe_zykluskosten:
    sheet_name = "var2_fix"
else:
    sheet_name = "var2_var"

# Prepare the solution data for export
solution_data = {
    "Hour": [h for h in H],
    "Einkauf": [],
    "Verkauf": [],
    "Risikoklasse": [],
    "Gebotspreis": [],
}

# Populate the solution_data dictionary
for h in H:
    v_h = 0
    e_h = 0
    risikoklasse_v = None
    risikoklasse_e = None
    for r in R:
        # Update the Verkauf and Einkauf values for the current hour
        v_h += scip.getVal(v_h_r[(h, r)])
        e_h += scip.getVal(e_h_r[(h, r)])
        # Determine the risk class for Verkauf and Einkauf
        if scip.getVal(v_h_r[(h, r)]) > 0:
            risikoklasse_v = r
        if scip.getVal(e_h_r[(h, r)]) > 0:
            risikoklasse_e = r

    # Append rounded values and risk classes to the solution data
    solution_data["Einkauf"].append(round(e_h, 3))
    solution_data["Verkauf"].append(round(v_h, 3))
    solution_data["Risikoklasse"].append(
        risikoklasse_v if round(v_h, 3) > 0 else risikoklasse_e if round(e_h, 3) > 0 else ""
    )
    solution_data["Gebotspreis"].append(
        gebotspreis_verkauf_h_r[(h, risikoklasse_v)] if round(v_h, 3) > 0 else gebotspreis_einkauf_h_r[(h, risikoklasse_e)] if round(e_h, 3) > 0 else ""
    )

# Convert to a DataFrame
solution_df = pd.DataFrame(solution_data)

# Write to Excel
try:
    # Load the workbook to check for existing worksheets
    workbook = load_workbook(file_path)
    if sheet_name not in workbook.sheetnames:
        # If the worksheet doesn't exist, create it
        print(f"Worksheet '{sheet_name}' not found. Creating it...")
        with pd.ExcelWriter(file_path, mode="a", engine="openpyxl") as writer:
            solution_df.to_excel(writer, sheet_name=sheet_name, index=False)
    else:
        # If the worksheet exists, overwrite it
        print(f"Worksheet '{sheet_name}' found. Writing solution...")
        with pd.ExcelWriter(file_path, mode="a", engine="openpyxl", if_sheet_exists="replace") as writer:
            solution_df.to_excel(writer, sheet_name=sheet_name, index=False)
except FileNotFoundError:
    # If the file doesn't exist, create it and write the solution
    print(f"File '{file_path}' not found. Creating it and writing solution...")
    with pd.ExcelWriter(file_path, mode="w", engine="openpyxl") as writer:
        solution_df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Solution written to '{file_path}' in the '{sheet_name}' worksheet.")


Worksheet 'var2_var' found. Writing solution...
Solution written to 'Ergebnisse.xlsx' in the 'var2_var' worksheet.
